In [7]:
import torch
from torch import nn

class SMRNN(nn.Module):
    def __init__(self,state_size,input_size,outputs) -> None:
        super().__init__()
        self.state = torch.zeros(state_size)
        self.ln1 = nn.Linear(input_size+state_size,state_size + outputs)
    
    def forward(self,inputs):
        y = torch.cat((self.state,inputs))
        x = self.ln1(y)
        self.state = x[:self.state.shape[0]]
        return x[self.state.shape[0]:]

    def cleanstate(self):
        self.state = torch.zeros_like(self.state)


In [11]:
import torch.nn.functional as F
from torch import optim

a = SMRNN(3,3,3)
opt = optim.SGD(a.parameters(),lr=0.01)
loss_func = F.mse_loss
epochs = 100
step_size = 5
for i in range(epochs):
    prev_inpt = torch.zeros(3)
    result = torch.zeros(3)
    for j in range(step_size):
        inpt = torch.rand(3)
        y = a.forward(inpt)
        result += inpt
        prev_inpt = inpt
    a.cleanstate()
    loss = loss_func(y,result)
    print(loss)
    loss.backward()
    opt.step()
    opt.zero_grad()

tensor([0.0000, 0.0000, 0.0000, 0.0530, 0.0077, 0.4969])
tensor(1.6915, grad_fn=<MseLossBackward0>)
tensor([-0.2261, -0.2998, -0.0138,  0.0570,  0.2944,  0.5091],
       grad_fn=<CatBackward0>)
tensor(1.3751, grad_fn=<MseLossBackward0>)
tensor([-0.3627, -0.4427, -0.0582,  0.1260,  0.1269,  0.2219],
       grad_fn=<CatBackward0>)
tensor(0.3898, grad_fn=<MseLossBackward0>)


RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [6, 6]], which is output 0 of AsStridedBackward0, is at version 3; expected version 2 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).